In [1]:
%run ../yp_utils.py

# Initial setup

In [2]:
paper_pmid = 22438835
paper_name = 'hoose_polymenis_2012' 

In [3]:
datasets = pd.read_csv('extras/YeastPhenome_' + str(paper_pmid) + '_datasets_list.txt', sep='\t', header=None, names=['dataset_id', 'name'])

In [4]:
datasets.set_index('dataset_id', inplace=True)

# Load & process the data

In [8]:
original_data = pd.read_excel('raw_data/Dataset_S1.xlsx', sheet_name='Data and Correlation', skiprows=1)

In [9]:
print('Original data dimensions: %d x %d' % (original_data.shape))

Original data dimensions: 4342 x 19


In [10]:
original_data.head()

,Plate ID / gene,ORF,# exp,%G1 (avg),%G1 (std),Mean FSC-H (avg),Mean FSC-H (std),Median FSC-H (avg),Median FSC-H (std),Growth coefficient,Mean,Median,Mean.1,Median.1,Mode,ORF.1,Common,Alias(es),Description
0,335B10,YDR004W,3,0.088046,0.030646,121.803333,13.366287,113.023333,12.199362,3.5,48.4691,43.9967,61.98,63.93,57.08,YDR004W,RAD57,NaN,Protein that stimulates strand exchange by sta...
1,302F01,YLR079W,7,0.103702,0.075621,130.492857,16.641394,125.784286,20.333919,7.8,48.1552,41.6297,53.53,55.87,25.11,YLR079W,SIC1,SDB25,Inhibitor of Cdc28-Clb kinase complexes that c...
2,344H08,YBR119W,4,0.116522,0.039177,113.107500,7.757016,105.770000,7.793857,1.0,43.1939,40.1944,61.37,66.36,65.29,YBR119W,MUD1,U1A U1-A,"U1 snRNP A protein, homolog of human U1-A; inv..."
3,330F02,YNL250W,3,0.120508,0.042201,162.136667,7.371651,160.326667,9.029354,8.4,57.2188,51.9686,67.08,72.58,47.95,YNL250W,RAD50,NaN,"Subunit of MRX complex, with Mre11p and Xrs2p,..."
4,317C01,YKL101W,3,0.125289,0.040510,149.546667,3.786031,143.620000,3.800776,1.0,55.2519,51.9686,61.56,65.53,59.81,YKL101W,HSL1,ELM2 NIK1,Nim1p-related protein kinase that regulates th...


In [11]:
original_data['orf'] = original_data['ORF'].astype(str)

In [12]:
# Eliminate all white spaces & capitalize
original_data['orf'] = clean_orf(original_data['orf'])

In [15]:
original_data.loc[original_data['orf']=='YELOO1C','orf'] = 'YEL001C'

In [16]:
# Translate to ORFs 
original_data['orf'] = translate_sc(original_data['orf'], to='orf')

In [17]:
# Make sure everything translated ok
t = looks_like_orf(original_data['orf'])
print(original_data.loc[~t,])

            Plate ID / gene     ORF  # exp  %G1 (avg)  %G1 (std)  \
index_input                                                        
1511                 BY4743     NaN    250   0.311694   0.051983   
3226                 304F06  YMR41W      2   0.364966   0.026440   

             Mean FSC-H (avg)  Mean FSC-H (std)  Median FSC-H (avg)  \
index_input                                                           
1511                 100.0894          6.789526            91.96456   
3226                  94.6250          2.213244            87.30000   

             Median FSC-H (std)  Growth coefficient  Mean  Median  Mean.1  \
index_input                                                                 
1511                   6.403580                 NaN   NaN     NaN     NaN   
3226                   1.400071                 1.0   NaN     NaN   44.09   

             Median.1   Mode   ORF.1 Common Alias(es) Description     orf  
index_input                                              

In [18]:
original_data = original_data.loc[t,:]

In [19]:
original_data['data'] = original_data['%G1 (avg)']

In [20]:
original_data.set_index('orf', inplace=True)

In [21]:
original_data = original_data[['data']].copy()

In [22]:
original_data = original_data.groupby(original_data.index).mean()

In [23]:
original_data.shape

(4240, 1)

# Prepare the final dataset

In [24]:
data = original_data.copy()

In [25]:
dataset_ids = [515]
datasets = datasets.reindex(index=dataset_ids)

In [26]:
lst = [datasets.index.values, ['value']*datasets.shape[0]]
tuples = list(zip(*lst))
idx = pd.MultiIndex.from_tuples(tuples, names=['dataset_id','data_type'])
data.columns = idx

In [27]:
data.head()

dataset_id,515
data_type,value
orf,
YAL002W,0.322576
YAL004W,0.321099
YAL005C,0.260488
YAL007C,0.315864
YAL008W,0.366145


## Subset to the genes currently in SGD

In [28]:
genes = pd.read_csv(path_to_genes, sep='\t', index_col='id')
genes = genes.reset_index().set_index('systematic_name')
gene_ids = genes.reindex(index=data.index.values)['id'].values
num_missing = np.sum(np.isnan(gene_ids))
print('ORFs missing from SGD: %d' % num_missing)

ORFs missing from SGD: 24


In [29]:
data['gene_id'] = gene_ids
data = data.loc[data['gene_id'].notnull()]
data['gene_id'] = data['gene_id'].astype(int)
data = data.reset_index().set_index(['gene_id','orf'])

data.head()

,dataset_id,515
,data_type,value
gene_id,orf,
2,YAL002W,0.322576
1863,YAL004W,0.321099
4,YAL005C,0.260488
5,YAL007C,0.315864
6,YAL008W,0.366145


# Normalize

In [30]:
data_norm = normalize_phenotypic_scores(data, has_tested=True)

In [31]:
# Assign proper column names
lst = [datasets.index.values, ['valuez']*datasets.shape[0]]
tuples = list(zip(*lst))
idx = pd.MultiIndex.from_tuples(tuples, names=['dataset_id','data_type'])
data_norm.columns = idx

In [32]:
data_norm[data.isnull()] = np.nan
data_all = data.join(data_norm)

data_all.head()

dataset_id            515          
data_type           value    valuez
gene_id orf                        
2       YAL002W  0.322576 -0.119306
1863    YAL004W  0.321099 -0.146495
4       YAL005C  0.260488 -1.261890
5       YAL007C  0.315864 -0.242832
6       YAL008W  0.366145  0.682484

# Print out

In [33]:
for f in ['value','valuez']:
    df = data_all.xs(f, level='data_type', axis=1).copy()
    df.columns = datasets['name'].values
    df = df.droplevel('gene_id', axis=0)
    df.to_csv(paper_name + '_' + f + '.txt', sep='\t')

# Save to DB

In [34]:
from IO.save_data_to_db3 import *

In [35]:
save_data_to_db(data_all, paper_pmid)

  0%|          | 0/1 [00:00<?, ?it/s]

Deleting all datasets for PMID 22438835...
Inserting the new data...


100%|██████████| 1/1 [00:06<00:00,  6.96s/it]

Updating the data_modified_on field...
